## Подготовка данных для занятия

In [1]:
import re

import pandas as pd
import pymorphy3


text_df = pd.read_csv("./data/content_description.csv", sep='\t')
text_df.head()

,content,description
0,https://www.ivi.ru/watch/157318/description,"Лучший подарок, который только можно было прид..."
1,https://www.ivi.ru/watch/98336/description,Через какие трудности приходится проходить Сан...
2,https://www.ivi.ru/watch/183533/description,Миловидный Давид - позор для своего отца. Не в...
3,https://www.ivi.ru/watch/157319/description,Экранизация сатирического бестселлера Стивена ...
4,https://www.ivi.ru/watch/51342/description,«Леди удача» – авантюрная романтическая комеди...


In [2]:
# разбиваем тест на слова
corpus = []
# регулярка для поиска слов
regular_expr = r'\w+'
reg_expr_compiled = re.compile(regular_expr)
# формируем датасет из отдельных слов
for raw_text in text_df.description.values:
    # приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    text_by_words = reg_expr_compiled.findall(raw_text_lower) 
    corpus.append(text_by_words)

corpus

[['лучший',
  'подарок',
  'который',
  'только',
  'можно',
  'было',
  'придумать',
  'для',
  'всех',
  'поклонников',
  'культового',
  'сериала',
  'рождественский',
  'спецвыпуск',
  'шерлока',
  'в',
  'котором',
  'абсолютно',
  'все',
  'будет',
  'особенным',
  'и',
  'старинные',
  'костюмы',
  'и',
  'дух',
  'викторианской',
  'англии',
  'и',
  'конечно',
  'же',
  'покрытая',
  'мраком',
  'и',
  'завесой',
  'мистики',
  'тайна',
  'которую',
  'предстоит',
  'разгадать',
  'гениальному',
  'сыщику',
  'кроме',
  'того',
  'создатели',
  'сериала',
  'обещали',
  'оставить',
  'для',
  'самых',
  'внимательных',
  'поклонников',
  'шерлока',
  'подсказки',
  'связанные',
  'с',
  'содержанием',
  'четвертого',
  'сезона',
  'сериала',
  'впервые',
  'герои',
  'современного',
  'сериала',
  'предстанут',
  'в',
  'своих',
  'книжных',
  'образах',
  'холмс',
  'с',
  'трубкой',
  'и',
  'в',
  'охотничьей',
  'шляпе',
  'а',
  'ватсон',
  'с',
  'усами',
  'и',
  'в',
 

In [3]:
# нормализация текста
normalized_corpus = []
morph = pymorphy3.MorphAnalyzer()
# нормализуем каждое слово в тексте
for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)

# превращаем в DataFrame
doc_count = len(normalized_corpus)
doc_ids = []
tokens = []
# формируем два списка-колонки датафрейма
for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
word_count_df = tokens_df.groupby(['doc_id','word'])['dummy'].count().reset_index()

word_count_df[word_count_df.doc_id==0].sort_values(by='dummy', ascending=False).head(10)

,doc_id,word,dummy
6,0,в,10
29,0,и,7
37,0,который,5
111,0,шерлок,4
77,0,с,4
84,0,сериал,4
75,0,риколетти,3
31,0,история,2
21,0,для,2
49,0,о,2


# 10.3  Домашняя работа

10.3.1 **Задание простого уровня** Для каждого слова подсчитайте процент документов, в которых содержится это слово. Сформируйте dataframe doc_frequency. Процен документов вычисляется по формуле
$$
d = \frac{m}{n} \times 100
$$
где $m$ - количество документов, в которых встретилось это слово, а $n$ - общее количество документов. Значение в процентах округлите до целых.

Посмотрите какие слова оказались в топе - это предлоги и частицы. Такие слова присутствуют во всех документах, а значит, их можно удалить из текста, чтобы оставшиеся слова были более "осмысленными". Это упростит анализ текста.

In [6]:
# -- ВАШ КОД ЗДЕСЬ --

doc_frequency = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
doc_frequency = doc_frequency[doc_frequency['word'].str.len() > 2]

word_count_df2 = doc_frequency.groupby(['word'])['doc_id'].count().reset_index()
word_count_df2.sort_values(by='doc_id', ascending=False).head(10)

,word,doc_id
460,свой,9
593,что,9
450,сам,8
308,они,8
209,который,8
186,как,6
451,самый,6
309,онлайн,6
86,год,6
478,смотреть,5


In [18]:
doc_frequency = pd.DataFrame()
doc_frequency = word_count_df.groupby(['word'])['doc_id'].count().reset_index(name='doc_freq').sort_values(by='doc_freq', ascending=False)
doc_frequency.doc_freq /= float(doc_count)
doc_frequency.head(20)

,word,doc_freq
171,и,1.000
41,в,1.000
269,на,0.875
468,с,0.875
323,он,0.750
480,свой,0.750
618,что,0.750
326,онлайн,0.750
283,не,0.750
90,год,0.625


10.3.2 **Задание среднего уровня**

Воспользуйтесь регулярными выражениями, чтобы извлечь из текста все пары **имя+фамилия**.

* неформальное описание регулярки: пара слов идущая друг за другом, каждое из которых начинается с заглавной буквы
* анализировать нужно только `doc_id=3`
* текст берём из исходного датафрейма `text_df`
* заглавная буква в русскоязычном тексте соответствует символьному классу `r'[A-Я]*'`

In [10]:
# -- ВАШ КОД ЗДЕСЬ --
raw_text = text_df.description.values[3]
print(raw_text)
# регулярка - её нужно поправить
reg_expr = r'[А-Я]\w+ [А-Я]\w+'
# компилируем регулярное выражение
reg_expr_compiled = re.compile(reg_expr)
# применяем выражение к тексту
for g in reg_expr_compiled.findall(raw_text):
    print(g)

Экранизация сатирического бестселлера Стивена Фрая, мастерски воплощенная британским режиссером и продюсером Джоном Дженксом.   У Теда Уоллеса было славное прошлое, и вот-вот его книги должны были пополнить стройный ряд томов британских классиков. Но его жуткий характер вкупе с алкоголизмом и мизантропией в запущенной стадии испортили все. Теперь Тед зарабатывает на жизнь тем, что поносит в своих статьях спектакли, которые вообще-то не стоят его внимания. И продолжает пить. Вскоре Тед лишается и этого: его бесконечное ворчание достало всю редакцию.  Готовясь к голодной и бесславной смерти, внезапно Уоллес получает выгодное предложение от своей крестницы Джейн. Девушка недавно излечилась от болезни благодаря чуду и теперь предлагает Теду погостить в аристократическом доме, чтобы осветить происходящие здесь чудеса исцеления.  Старый скептик соглашается из корыстных побуждений и даже представить не может, сколько сюрпризов ему уготовано.  Смотреть онлайн эту комедию о том, что даже самые 

10.3.3 **Задание сложного уровня**. Словарь `genre_dict` содержит слова, которые являются характерными для того или иного жанра. Пользуясь словарём, а так же таблицей `word_count_df`, сформируйте таблицу двумя колонками `doc_id | genre` с жанрами фильмов.

Для этого нужно
* превратить словарь `genre_dict` в DataFrame формата `word | genre`
* соединить полученный датафрейм с помощью функции `merge` c ранее полученным датафреймом `word_count_df`, котороый содержит распределение слов по документам. Воспользуйтесь методом соединения `inner`
* для каждого документа выбрать жанр документа - это совокупность жанров слов отдельных слов. У одного контента может быть несколько жанров

In [16]:
import itertools
import pandas as pd

genre_dict = {
    'комедия': ['сатирический', 'авантюрный', 'забавный'],
    'мелодрама': ['выбор', 'позор'],
    'сказка': ['приключения', 'милый', 'семейный'],
    'детектив': ['тайна', 'разгадать', 'загадочный'], 
    'триллер': ['ужас', 'зловещий', 'нерв']
}

nested_genres =[[(i, j) for j in genre_dict[i]] for i in genre_dict]
# переводим из словаря к более удобному виду
flatten_genres = list(itertools.chain(*nested_genres))
# создаём DataFrame
genres_df = pd.DataFrame(flatten_genres, columns = ['genre', 'character_word'])

# -- ВАШ КОД ЗДЕСЬ --
# doc_to_genres = pd.DataFrame()
doc_to_genres = pd.merge(word_count_df, genres_df, left_on='word', right_on='character_word').groupby('doc_id')
doc_to_genres.head()

,doc_id,word,dummy,genre,character_word
0,0,загадочный,1,детектив,загадочный
1,0,разгадать,1,детектив,разгадать
2,0,тайна,1,детектив,тайна
3,0,ужас,1,триллер,ужас
4,1,милый,1,сказка,милый
5,2,выбор,1,мелодрама,выбор
6,2,позор,1,мелодрама,позор
7,3,сатирический,1,комедия,сатирический
8,4,авантюрный,1,комедия,авантюрный
9,4,забавный,1,комедия,забавный
